In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'P100'
file_name = 'P100'

In [3]:
DATA_DIR = os.path.abspath("../../../prediction_model/data/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model/data/")

## Dense Layer

In [4]:
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/2/benchmark_dense__20181119.pkl' %file_name))

for i in range(3,3):
    dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20181119.pkl' %(file_name, i)))])

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [7]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,32.784080,2037.468080,2042.244520,0.0,32.0,0.732888,0.701663,0.679047,0.019520,1.365308e+08,...,0.083280,0.083080,0.084800,0.493120,0.085440,0.085040,0.251960,0.249560,0.251560,0.246920
std,18.460976,1187.081709,1183.780913,0.0,0.0,0.395866,0.391203,0.392509,0.009891,1.597323e+08,...,0.276311,0.276009,0.278589,0.499963,0.279541,0.278947,0.434147,0.432767,0.433918,0.431228
min,1.000000,1.000000,1.000000,0.0,32.0,0.346479,0.326743,0.274577,0.000976,7.600000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,996.000000,1011.000000,0.0,32.0,0.456280,0.428758,0.402660,0.012755,2.204244e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,2043.000000,2035.000000,0.0,32.0,0.592036,0.558889,0.538108,0.017729,7.419462e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,3065.250000,3077.250000,0.0,32.0,0.845661,0.812736,0.792259,0.024069,1.949631e+08,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,64.000000,4095.000000,4095.000000,0.0,32.0,3.094482,3.024440,2.880960,0.117104,1.031366e+09,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [9]:
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/0/benchmark_convolution__20181030.pkl' %file_name))

header = dfConv.columns

for i in range(1,2):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%i/benchmark_convolution__20181101.pkl' %(file_name, i)))])

In [10]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [11]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [12]:
dfConv.describe()

,batchsize,channels_in,channels_out,gpu,kernelsize,matsize,padding,precision,strides,timeUsed_max,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.00000,25000.0,25000.00000,25000.000000,25000.000000,25000.0,25000.000000,25000.000000,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,24.558640,66.293040,67.09828,0.0,3.97396,256.578560,0.502480,32.0,2.493840,10.503715,...,0.085280,0.084320,0.081200,0.503560,0.080640,0.083280,0.248640,0.253720,0.246040,0.251600
std,13.832315,316.782091,342.48956,0.0,1.99872,148.098369,0.500004,0.0,1.122075,17.400537,...,0.279303,0.277873,0.273147,0.499997,0.272287,0.276311,0.432233,0.435148,0.430711,0.433941
min,1.000000,1.000000,1.00000,0.0,1.00000,1.000000,0.000000,32.0,1.000000,0.195780,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,10.000000,10.00000,0.0,2.00000,128.000000,0.000000,32.0,1.000000,1.565231,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,25.000000,19.000000,19.00000,0.0,4.00000,256.000000,1.000000,32.0,2.000000,4.171820,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,39.000000,38.00000,0.0,6.00000,386.000000,1.000000,32.0,4.000000,11.618330,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,48.000000,9942.000000,9908.00000,0.0,7.00000,512.000000,1.000000,32.0,4.000000,255.925422,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))